In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
nn_aqi_df = pd.read_csv("AirQualityAndWeatherDataMerged.csv")
nn_aqi_df.head()

,Unnamed: 0,state,county,city,date_local,parameter,sample_duration,units_of_measure,arithmetic_mean,aqi,...,Precipitation,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions
0,0,Tennessee,Sumner,Hendersonville,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),6.2,26.0,...,0.00,0.0,0.0,13.4,292.13,19.7,9.9,32.2,58.99,Partially cloudy
1,1,Tennessee,Davidson,Nashville,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.1,21.0,...,0.00,0.0,0.0,13.9,278.04,21.9,9.9,32.1,57.18,Partially cloudy
2,2,Tennessee,Davidson,Nashville,2018-01-02,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),3.8,16.0,...,0.00,0.0,0.0,8.8,153.88,NaN,9.9,24.8,50.71,Clear
3,3,Tennessee,Sumner,Hendersonville,2018-01-02,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.6,23.0,...,0.00,0.0,0.0,7.9,166.08,NaN,9.9,18.0,52.93,Clear
4,4,Tennessee,Sumner,Hendersonville,2018-01-03,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.3,22.0,...,0.01,0.0,0.0,11.9,252.83,24.2,9.9,60.1,50.06,"Rain, Partially cloudy"


In [2]:
# Binning aqi values
aqi_bins = [0, 50, 100, 300, 500]
group_names = ['Good', 'Moderate', 'Unhealthy', 'Hazardous']

# Categorize aqi based on the bins.
nn_aqi_df["AQI_Range"] = pd.cut(nn_aqi_df["aqi"], aqi_bins, labels=group_names)

nn_aqi_df

,Unnamed: 0,state,county,city,date_local,parameter,sample_duration,units_of_measure,arithmetic_mean,aqi,...,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions,AQI_Range
0,0,Tennessee,Sumner,Hendersonville,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),6.2,26.0,...,0.0,0.0,13.4,292.13,19.7,9.9,32.2,58.99,Partially cloudy,Good
1,1,Tennessee,Davidson,Nashville,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.1,21.0,...,0.0,0.0,13.9,278.04,21.9,9.9,32.1,57.18,Partially cloudy,Good
2,2,Tennessee,Davidson,Nashville,2018-01-02,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),3.8,16.0,...,0.0,0.0,8.8,153.88,NaN,9.9,24.8,50.71,Clear,Good
3,3,Tennessee,Sumner,Hendersonville,2018-01-02,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.6,23.0,...,0.0,0.0,7.9,166.08,NaN,9.9,18.0,52.93,Clear,Good
4,4,Tennessee,Sumner,Hendersonville,2018-01-03,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.3,22.0,...,0.0,0.0,11.9,252.83,24.2,9.9,60.1,50.06,"Rain, Partially cloudy",Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,1667,Tennessee,Davidson,Nashville,2020-12-30,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),8.7,36.0,...,0.0,0.0,23.8,170.63,38.1,9.3,68.3,63.09,"Rain, Partially cloudy",Good
1668,1668,Tennessee,Sumner,Hendersonville,2020-12-30,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),7.3,30.0,...,0.0,0.0,22.1,185.75,36.1,9.3,67.1,64.16,"Rain, Partially cloudy",Good
1669,1669,Tennessee,Sumner,Hendersonville,2020-12-31,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),1.0,4.0,...,0.0,0.0,16.8,127.13,21.8,7.7,97.0,91.89,"Rain, Overcast",Good
1670,1670,Tennessee,Davidson,Nashville,2020-12-31,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),2.2,9.0,...,0.0,0.0,17.4,128.83,25.0,7.6,98.8,90.33,"Rain, Overcast",Good


In [3]:
nn_aqi_df["AQI_Range"] = nn_aqi_df["AQI_Range"].astype('category').cat.codes

In [4]:
nn_aqi_df.dtypes

Unnamed: 0               int64
state                   object
county                  object
city                    object
date_local              object
parameter               object
sample_duration         object
units_of_measure        object
arithmetic_mean        float64
aqi                    float64
City                    object
State                   object
Country                 object
Date time               object
Maximum Temperature    float64
Minimum Temperature    float64
Temperature            float64
Wind Chill             float64
Heat Index             float64
Precipitation          float64
Snow                   float64
Snow Depth             float64
Wind Speed             float64
Wind Direction         float64
Wind Gust              float64
Visibility             float64
Cloud Cover            float64
Relative Humidity      float64
Conditions              object
AQI_Range                 int8
dtype: object

In [5]:
# Generate our categorical variable lists
nn_aqi_cat = nn_aqi_df.dtypes[nn_aqi_df.dtypes == "object"].index.tolist()
nn_aqi_cat

['state',
 'county',
 'city',
 'date_local',
 'parameter',
 'sample_duration',
 'units_of_measure',
 'City',
 'State',
 'Country',
 'Date time',
 'Conditions']

In [6]:
# Drop the non-beneficial columns, "Unnamed: 0", "parameter", "sample_duration", "units_of_measure", "state", "county", "Date time", "City", "State", "Country", "arithmetic_mean", "date_local", "city" 
nn_aqi_df = nn_aqi_df.drop(columns=["Unnamed: 0", "parameter", "sample_duration", "units_of_measure", "state", "county", "Date time", "City", "State", "Country", "arithmetic_mean", "date_local", "city", "aqi" ])
nn_aqi_df

,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions,AQI_Range
0,21.1,6.8,13.3,-7.5,NaN,0.00,0.0,0.0,13.4,292.13,19.7,9.9,32.2,58.99,Partially cloudy,0
1,21.7,7.1,13.6,-7.9,NaN,0.00,0.0,0.0,13.9,278.04,21.9,9.9,32.1,57.18,Partially cloudy,0
2,25.8,4.8,14.6,-6.9,NaN,0.00,0.0,0.0,8.8,153.88,NaN,9.9,24.8,50.71,Clear,0
3,25.0,4.3,13.9,-6.8,NaN,0.00,0.0,0.0,7.9,166.08,NaN,9.9,18.0,52.93,Clear,0
4,34.6,16.4,24.5,9.9,NaN,0.01,0.0,0.0,11.9,252.83,24.2,9.9,60.1,50.06,"Rain, Partially cloudy",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,66.7,35.4,56.0,28.9,NaN,0.12,0.0,0.0,23.8,170.63,38.1,9.3,68.3,63.09,"Rain, Partially cloudy",0
1668,66.1,36.2,55.5,30.0,NaN,0.25,0.0,0.0,22.1,185.75,36.1,9.3,67.1,64.16,"Rain, Partially cloudy",0
1669,45.2,38.6,41.0,31.8,NaN,0.90,0.0,0.0,16.8,127.13,21.8,7.7,97.0,91.89,"Rain, Overcast",0
1670,46.3,39.1,41.6,32.3,NaN,0.86,0.0,0.0,17.4,128.83,25.0,7.6,98.8,90.33,"Rain, Overcast",0


In [7]:
# Determine the number of unique values in each column.
nn_aqi_df.nunique()

Maximum Temperature     536
Minimum Temperature     525
Temperature             536
Wind Chill              316
Heat Index              208
Precipitation           136
Snow                     15
Snow Depth               18
Wind Speed              202
Wind Direction         1370
Wind Gust               226
Visibility               56
Cloud Cover             760
Relative Humidity      1295
Conditions                9
AQI_Range                 4
dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = nn_aqi_df.dtypes[nn_aqi_df.dtypes == "object"].index.tolist()
application_cat

['Conditions']

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(nn_aqi_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,Conditions_Clear,Conditions_Overcast,Conditions_Partially cloudy,Conditions_Rain,"Conditions_Rain, Overcast","Conditions_Rain, Partially cloudy",Conditions_Snow,"Conditions_Snow, Overcast","Conditions_Snow, Partially cloudy"
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
nn_aqi_df = nn_aqi_df.merge(encode_df, left_index=True,right_index=True)
nn_aqi_df = nn_aqi_df.drop(columns = application_cat)
nn_aqi_df.head()

,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow,Snow Depth,Wind Speed,Wind Direction,...,AQI_Range,Conditions_Clear,Conditions_Overcast,Conditions_Partially cloudy,Conditions_Rain,"Conditions_Rain, Overcast","Conditions_Rain, Partially cloudy",Conditions_Snow,"Conditions_Snow, Overcast","Conditions_Snow, Partially cloudy"
0,21.1,6.8,13.3,-7.5,NaN,0.00,0.0,0.0,13.4,292.13,...,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21.7,7.1,13.6,-7.9,NaN,0.00,0.0,0.0,13.9,278.04,...,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,25.8,4.8,14.6,-6.9,NaN,0.00,0.0,0.0,8.8,153.88,...,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,25.0,4.3,13.9,-6.8,NaN,0.00,0.0,0.0,7.9,166.08,...,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,34.6,16.4,24.5,9.9,NaN,0.01,0.0,0.0,11.9,252.83,...,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [11]:
# Determine the number of unique values in each column.
nn_aqi_df.nunique()

Maximum Temperature                   536
Minimum Temperature                   525
Temperature                           536
Wind Chill                            316
Heat Index                            208
Precipitation                         136
Snow                                   15
Snow Depth                             18
Wind Speed                            202
Wind Direction                       1370
Wind Gust                             226
Visibility                             56
Cloud Cover                           760
Relative Humidity                    1295
AQI_Range                               4
Conditions_Clear                        2
Conditions_Overcast                     2
Conditions_Partially cloudy             2
Conditions_Rain                         2
Conditions_Rain, Overcast               2
Conditions_Rain, Partially cloudy       2
Conditions_Snow                         2
Conditions_Snow, Overcast               2
Conditions_Snow, Partially cloudy 

In [12]:
# Drop the non-beneficial columns, "Maximum Temperature", "Minimum Temperature", "Wind Chill", "Snow Depth", "Wind Gust", "Wind Direction" 
nn_aqi_df = nn_aqi_df.drop(columns=["Maximum Temperature", "Minimum Temperature", "Wind Chill", "Snow Depth", "Wind Gust", "Heat Index", "Wind Direction", "Precipitation", "Snow", "Visibility", "Cloud Cover", "Relative Humidity" ])
nn_aqi_df

,Temperature,Wind Speed,AQI_Range,Conditions_Clear,Conditions_Overcast,Conditions_Partially cloudy,Conditions_Rain,"Conditions_Rain, Overcast","Conditions_Rain, Partially cloudy",Conditions_Snow,"Conditions_Snow, Overcast","Conditions_Snow, Partially cloudy"
0,13.3,13.4,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.6,13.9,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14.6,8.8,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13.9,7.9,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24.5,11.9,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1667,56.0,23.8,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1668,55.5,22.1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1669,41.0,16.8,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1670,41.6,17.4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = nn_aqi_df["AQI_Range"].values
X = nn_aqi_df.drop("AQI_Range", 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [14]:
print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (1254, 11) (1254,)
Test (418, 11) (418,)


In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  75
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 75)                900       
_________________________________________________________________
dense_5 (Dense)              (None, 50)                3800      
_________________________________________________________________
dense_6 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 26        
Total params: 6,001
Trainable params: 6,001
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=500)

Epoch 1/500
40/40 [==============================] - 0s 896us/step - loss: 0.3854 - accuracy: 0.8740
Epoch 2/500
40/40 [==============================] - 0s 869us/step - loss: 0.3180 - accuracy: 0.8939
Epoch 3/500
40/40 [==============================] - 0s 844us/step - loss: 0.3161 - accuracy: 0.8939
Epoch 4/500
40/40 [==============================] - 0s 895us/step - loss: 0.3129 - accuracy: 0.8939
Epoch 5/500
40/40 [==============================] - 0s 844us/step - loss: 0.3050 - accuracy: 0.8939
Epoch 6/500
40/40 [==============================] - 0s 895us/step - loss: 0.3272 - accuracy: 0.8939
Epoch 7/500
40/40 [==============================] - 0s 895us/step - loss: 0.3272 - accuracy: 0.8939
Epoch 8/500
40/40 [==============================] - 0s 844us/step - loss: 0.3067 - accuracy: 0.8939
Epoch 9/500
40/40 [==============================] - 0s 844us/step - loss: 0.3124 - accuracy: 0.8939
Epoch 10/500
40/40 [==============================] - 0s 869us/step - loss: 0.3047 - accura

40/40 [==============================] - 0s 869us/step - loss: 0.2918 - accuracy: 0.8947
Epoch 82/500
40/40 [==============================] - 0s 946us/step - loss: 0.2965 - accuracy: 0.8947
Epoch 83/500
40/40 [==============================] - 0s 844us/step - loss: 0.2896 - accuracy: 0.8939
Epoch 84/500
40/40 [==============================] - 0s 793us/step - loss: 0.2898 - accuracy: 0.8955
Epoch 85/500
40/40 [==============================] - 0s 767us/step - loss: 0.2915 - accuracy: 0.8955
Epoch 86/500
40/40 [==============================] - 0s 767us/step - loss: 0.2892 - accuracy: 0.8947
Epoch 87/500
40/40 [==============================] - 0s 792us/step - loss: 0.2892 - accuracy: 0.8955
Epoch 88/500
40/40 [==============================] - 0s 793us/step - loss: 0.2958 - accuracy: 0.8947
Epoch 89/500
40/40 [==============================] - 0s 742us/step - loss: 0.2878 - accuracy: 0.8955
Epoch 90/500
40/40 [==============================] - 0s 767us/step - loss: 0.2894 - accuracy: 

40/40 [==============================] - 0s 801us/step - loss: 0.2815 - accuracy: 0.8955
Epoch 161/500
40/40 [==============================] - 0s 793us/step - loss: 0.2830 - accuracy: 0.8955
Epoch 162/500
40/40 [==============================] - 0s 793us/step - loss: 0.2844 - accuracy: 0.8955
Epoch 163/500
40/40 [==============================] - 0s 793us/step - loss: 0.2854 - accuracy: 0.8955
Epoch 164/500
40/40 [==============================] - 0s 818us/step - loss: 0.2833 - accuracy: 0.8955
Epoch 165/500
40/40 [==============================] - 0s 767us/step - loss: 0.2840 - accuracy: 0.8955
Epoch 166/500
40/40 [==============================] - 0s 793us/step - loss: 0.2799 - accuracy: 0.8955
Epoch 167/500
40/40 [==============================] - 0s 818us/step - loss: 0.2823 - accuracy: 0.8955
Epoch 168/500
40/40 [==============================] - 0s 767us/step - loss: 0.2810 - accuracy: 0.8955
Epoch 169/500
40/40 [==============================] - 0s 818us/step - loss: 0.2804 - a

40/40 [==============================] - 0s 793us/step - loss: 0.2838 - accuracy: 0.8955
Epoch 240/500
40/40 [==============================] - 0s 818us/step - loss: 0.2758 - accuracy: 0.8955
Epoch 241/500
40/40 [==============================] - 0s 818us/step - loss: 0.2742 - accuracy: 0.8955
Epoch 242/500
40/40 [==============================] - 0s 793us/step - loss: 0.2798 - accuracy: 0.8955
Epoch 243/500
40/40 [==============================] - 0s 793us/step - loss: 0.2751 - accuracy: 0.8955
Epoch 244/500
40/40 [==============================] - 0s 767us/step - loss: 0.2794 - accuracy: 0.8955
Epoch 245/500
40/40 [==============================] - 0s 793us/step - loss: 0.2748 - accuracy: 0.8955
Epoch 246/500
40/40 [==============================] - 0s 767us/step - loss: 0.2745 - accuracy: 0.8955
Epoch 247/500
40/40 [==============================] - 0s 767us/step - loss: 0.2755 - accuracy: 0.8955
Epoch 248/500
40/40 [==============================] - 0s 767us/step - loss: 0.2791 - a

40/40 [==============================] - 0s 793us/step - loss: 0.2590 - accuracy: 0.8955
Epoch 319/500
40/40 [==============================] - 0s 818us/step - loss: 0.2676 - accuracy: 0.8955
Epoch 320/500
40/40 [==============================] - 0s 793us/step - loss: 0.2647 - accuracy: 0.8955
Epoch 321/500
40/40 [==============================] - 0s 767us/step - loss: 0.2580 - accuracy: 0.8955
Epoch 322/500
40/40 [==============================] - 0s 793us/step - loss: 0.2674 - accuracy: 0.8963
Epoch 323/500
40/40 [==============================] - 0s 767us/step - loss: 0.2781 - accuracy: 0.8947
Epoch 324/500
40/40 [==============================] - 0s 767us/step - loss: 0.2649 - accuracy: 0.8947
Epoch 325/500
40/40 [==============================] - 0s 767us/step - loss: 0.2626 - accuracy: 0.8955
Epoch 326/500
40/40 [==============================] - 0s 793us/step - loss: 0.2613 - accuracy: 0.8955
Epoch 327/500
40/40 [==============================] - 0s 793us/step - loss: 0.2672 - a

40/40 [==============================] - 0s 742us/step - loss: 0.2304 - accuracy: 0.8987
Epoch 398/500
40/40 [==============================] - 0s 819us/step - loss: 0.2299 - accuracy: 0.8963
Epoch 399/500
40/40 [==============================] - 0s 793us/step - loss: 0.2135 - accuracy: 0.8987
Epoch 400/500
40/40 [==============================] - 0s 767us/step - loss: 0.2162 - accuracy: 0.8995
Epoch 401/500
40/40 [==============================] - 0s 793us/step - loss: 0.2520 - accuracy: 0.8987
Epoch 402/500
40/40 [==============================] - 0s 767us/step - loss: 0.2111 - accuracy: 0.8979
Epoch 403/500
40/40 [==============================] - 0s 793us/step - loss: 0.2325 - accuracy: 0.9027
Epoch 404/500
40/40 [==============================] - 0s 793us/step - loss: 0.2180 - accuracy: 0.8971
Epoch 405/500
40/40 [==============================] - 0s 767us/step - loss: 0.2073 - accuracy: 0.8995
Epoch 406/500
40/40 [==============================] - 0s 793us/step - loss: 0.2413 - a

40/40 [==============================] - 0s 793us/step - loss: 0.0682 - accuracy: 0.8987
Epoch 477/500
40/40 [==============================] - 0s 793us/step - loss: 0.1257 - accuracy: 0.8979
Epoch 478/500
40/40 [==============================] - 0s 793us/step - loss: 0.1103 - accuracy: 0.8995
Epoch 479/500
40/40 [==============================] - 0s 793us/step - loss: 0.0493 - accuracy: 0.8979
Epoch 480/500
40/40 [==============================] - 0s 793us/step - loss: 0.0890 - accuracy: 0.8971
Epoch 481/500
40/40 [==============================] - 0s 793us/step - loss: 0.0362 - accuracy: 0.8987
Epoch 482/500
40/40 [==============================] - 0s 818us/step - loss: 0.0850 - accuracy: 0.8971
Epoch 483/500
40/40 [==============================] - 0s 783us/step - loss: 0.0354 - accuracy: 0.9011
Epoch 484/500
40/40 [==============================] - 0s 793us/step - loss: 0.0727 - accuracy: 0.9003
Epoch 485/500
40/40 [==============================] - 0s 793us/step - loss: 0.0530 - a

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

14/14 - 0s - loss: 1.5753 - accuracy: 0.7297
Loss: 1.5753010511398315, Accuracy: 0.7296651005744934
